<a href="https://colab.research.google.com/github/fabiobaroliveira/gerador_base_de_dados/blob/main/locadora_de_ve%C3%ADculos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Criando base de dados de uma locadora de veículos

---
- Clientes
- Locações
- Frota de veículos


In [1]:
#Necessário instalar a biblioteca faker
! pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 30.3 MB/s eta 0:00:00


In [22]:
import pandas as pd
import random
from faker import Faker
from datetime import datetime, timedelta

fake = Faker('pt_BR')

# Parâmetros
num_clientes = 100
num_veiculos = 50
num_lojas = 5
num_locacoes = 555

# Gerar Clientes
clientes = []
for i in range(num_clientes):
    clientes.append({
        'id_cliente': i + 1,
        'nome': fake.name(),
        'cpf': fake.unique.cpf(),
        'email': fake.email(),
        'telefone': fake.phone_number(),
        'nps': random.choices([10, 9, 8, 7, 6, 5, 4], weights=[0.3, 0.3, 0.2, 0.05, 0.05, 0.05, 0.05])[0]
    })
clientes_df = pd.DataFrame(clientes)

# Gerar Veículos

# Categorias e faixas de preço por dia
categorias = {
    'Econômico': (98, 157),
    'Intermediário': (180, 247),
    'SUV': (299, 433)
}

# Marcas, modelos e categorias
marcas_modelos_categorias = [
    ('Fiat', 'Argo', 'Econômico'),
    ('Chevrolet', 'Onix', 'Econômico'),
    ('Volkswagen', 'Gol', 'Econômico'),
    ('Hyundai', 'HB20', 'Intermediário'),
    ('Ford', 'Ka', 'Econômico'),
    ('Toyota', 'Corolla', 'Intermediário'),
    ('Jeep', 'Renegade', 'SUV'),
    ('Honda', 'HR-V', 'SUV')
]

veiculos = []
for i in range(num_veiculos):
    marca, modelo, categoria = random.choice(marcas_modelos_categorias)
    ano = random.randint(2018, 2023)
    veiculos.append({
        'id_veiculo': i + 1,
        'marca': marca,
        'modelo': modelo,
        'categoria': categoria,
        'ano': ano,
        'placa': fake.unique.license_plate(),
        'status': 'disponível',
        'km_total': random.randint(20000, 150000),
        'tempo_manutencao_dias': random.randint(0, 20)
    })
veiculos_df = pd.DataFrame(veiculos)

# Gerar Lojas
cidades_estados = { "São Paulo": "SP",
                  "Rio de Janeiro": "RJ",
                  "Belo Horizonte": "MG",
                  "Porto Alegre": "RS",
                  "Curitiba": "PR"}

lojas = []
for i, (cidade, estado) in enumerate(cidades_estados.items(), start=1):
    lojas.append({
        "id_loja": i,
        "nome_loja": f"Locadora {cidade}",
        "cidade": cidade,
        "estado": estado
    })

lojas_df = pd.DataFrame(lojas)

# Gerar Locações
locacoes = []
historico_veiculos = {v: [] for v in veiculos_df['id_veiculo'].tolist()}
tentativas = 0
while len(locacoes) < num_locacoes and tentativas < num_locacoes * 10:
    tentativas += 1
    id_cliente = random.choice(clientes_df['id_cliente'].tolist())
    id_veiculo = random.choice(veiculos_df['id_veiculo'].tolist())
    id_loja = random.choice(lojas_df['id_loja'].tolist())
    data_inicio = fake.date_between(start_date='-2y', end_date='today')
    duracao = random.randint(1, 15)
    data_fim = data_inicio + timedelta(days=duracao)
    if random.random() < 0.2:
        data_fim = None  # locação ativa

    conflito = False
    for loc in historico_veiculos[id_veiculo]:
        inicio = loc['data_inicio']
        fim = loc['data_fim'] if loc['data_fim'] else datetime.today().date()
        if data_fim:
            if not (data_fim <= inicio or data_inicio >= fim):
                conflito = True
                break
        else:
            if data_inicio <= fim:
                conflito = True
                break

    if not conflito:
        categoria = veiculos_df.loc[veiculos_df['id_veiculo'] == id_veiculo, 'categoria'].values[0]
        preco_min, preco_max = categorias[categoria]
        preco_dia = random.randint(preco_min, preco_max)
        receita = preco_dia * duracao if data_fim else 0
        custo = receita * random.uniform(0.4, 0.7)
        km_rodado = random.randint(100, 1000)
        reclamacao = random.choices([0, 1], weights=[0.9, 0.1])[0]
        sinistro = random.choices([0, 1], weights=[0.95, 0.05])[0]

        nova_loc = {
            'id_locacao': len(locacoes) + 1,
            'id_cliente': id_cliente,
            'id_veiculo': id_veiculo,
            'id_loja': id_loja,
            'data_inicio': data_inicio,
            'data_fim': data_fim,
            'receita': receita,
            'custo': custo,
            'km_rodado': km_rodado,
            'reclamacao': reclamacao,
            'sinistro': sinistro,
            'preco_dia': preco_dia
        }
        locacoes.append(nova_loc)
        historico_veiculos[id_veiculo].append(nova_loc)

locacoes_df = pd.DataFrame(locacoes)

# Vendas e Marketing
marketing = []

for mes in pd.date_range(start='2023-01-01', end='2024-12-01', freq='MS'):
    cotacoes = random.randint(150, 400)
    alugueis_confirmados = random.randint(int(cotacoes * 0.3), cotacoes)
    investimento_marketing = random.randint(5000, 15000)
    novos_clientes = random.randint(20, 80)

    taxa_conversao = alugueis_confirmados / cotacoes
    cac = investimento_marketing / novos_clientes if novos_clientes else 0

    marketing.append({
        'mes': mes.strftime('%Y-%m'),
        'cotacoes': cotacoes,
        'alugueis_confirmados': alugueis_confirmados,
        'taxa_conversao': round(taxa_conversao, 2),
        'investimento_marketing': investimento_marketing,
        'novos_clientes': novos_clientes,
        'cac': round(cac, 2)
    })

marketing_df = pd.DataFrame(marketing)

In [23]:
# Exibir amostras
print(clientes_df.head())
print(veiculos_df.head())
print(lojas_df.head())
print(locacoes_df.head())
print(marketing_df.head())

   id_cliente             nome             cpf                         email  \
0           1   Ana Liz Novais  298.046.375-29           zguerra@example.com   
1           2      Nina Macedo  714.328.560-90         cauerocha@example.net   
2           3  Alexandre Moura  852.041.976-30            luan26@example.com   
3           4    Mariane Gomes  576.034.981-39        borgeseloa@example.com   
4           5    Renan Marques  380.271.546-26  montenegroalexia@example.org   

           telefone  nps  
0  +55 31 8260 5564   10  
1     0500-165-0517    9  
2   (081) 9497 9934   10  
3   (061) 0811-2954    8  
4      61 3417-9765    9  
   id_veiculo      marca   modelo      categoria   ano     placa      status  \
0           1     Toyota  Corolla  Intermediário  2019  XGX-6Q29  disponível   
1           2  Chevrolet     Onix      Econômico  2022  TVY-3I79  disponível   
2           3      Honda     HR-V            SUV  2018  KNQ-4H25  disponível   
3           4       Fiat     Argo    

In [17]:
clientes_df.head()

,id_cliente,nome,cpf,email,telefone,nps
0,1,Mariana da Paz,821.549.637-73,antoniomachado@example.org,0500 361 7054,9
1,2,João Vitor Novais,816.437.925-55,juliana98@example.net,+55 (051) 6817-7150,9
2,3,Carlos Eduardo Silva,792.614.358-82,esiqueira@example.org,11 3765 6434,9
3,4,Yasmin Santos,510.328.476-62,mmacedo@example.net,+55 (061) 7800-1115,8
4,5,Vinícius Macedo,943.128.750-14,cauapacheco@example.org,(051) 0219 0960,5


In [18]:
veiculos_df.head()

,id_veiculo,marca,modelo,categoria,ano,placa,status,km_total,tempo_manutencao_dias
0,1,Volkswagen,Gol,Econômico,2019,SNF-4P40,disponível,92348,14
1,2,Honda,HR-V,SUV,2020,NHH-4L96,disponível,36280,19
2,3,Chevrolet,Onix,Econômico,2019,ISP-7O20,disponível,39417,11
3,4,Toyota,Corolla,Intermediário,2023,ZHU-5I70,disponível,93650,0
4,5,Toyota,Corolla,Intermediário,2021,MKM-8N07,disponível,54896,10


In [19]:
lojas_df.head()

,id_loja,nome_loja,cidade,estado
0,1,Locadora São Paulo,São Paulo,SP
1,2,Locadora Rio de Janeiro,Rio de Janeiro,RJ
2,3,Locadora Belo Horizonte,Belo Horizonte,MG
3,4,Locadora Porto Alegre,Porto Alegre,RS
4,5,Locadora Curitiba,Curitiba,PR


In [20]:
locacoes_df.head()

,id_locacao,id_cliente,id_veiculo,id_loja,data_inicio,data_fim,receita,custo,km_rodado,reclamacao,sinistro,preco_dia
0,1,20,3,2,2024-05-13,None,0,0.000000,422,0,0,150
1,2,95,21,5,2024-09-06,2024-09-08,656,385.666051,818,1,0,328
2,3,45,35,4,2025-03-09,2025-03-19,1140,573.608717,260,0,0,114
3,4,60,49,3,2024-12-12,2024-12-26,1512,830.960687,628,0,0,108
4,5,99,50,1,2023-08-16,2023-08-23,819,555.029214,827,0,0,117


In [24]:
marketing_df.head()

,mes,cotacoes,alugueis_confirmados,taxa_conversao,investimento_marketing,novos_clientes,cac
0,2023-01,196,142,0.72,12661,64,197.83
1,2023-02,360,131,0.36,7607,63,120.75
2,2023-03,254,179,0.70,13689,29,472.03
3,2023-04,163,107,0.66,14120,45,313.78
4,2023-05,274,141,0.51,9906,64,154.78
